# UCD MSBA COVID-19 CHALLENGE
##### Team Disinfectant Wipes
##### 2020-04-15

## Data Collection: google_trends API
- Recommend to run the model using the final_dataset.csv file because the API could run for a while
- Recomment to run this part in colab if needed

In [4]:
#!pip install pytrends
import pandas as pd    
import numpy as np
from pytrends.request import TrendReq
import io
from google.colab import files

In [4]:
pytrends = TrendReq()   #connecting to google
uploaded = files.upload()
state = pd.read_csv(io.StringIO(uploaded['us_state.csv'].decode('utf-8')))
state.head()

Saving us_state.csv to us_state.csv


,State,Abbrev,Code,Country,geo
0,Alabama,Ala.,AL,US,US-AL
1,Alaska,Alaska,AK,US,US-AK
2,Arizona,Ariz.,AZ,US,US-AZ
3,Arkansas,Ark.,AR,US,US-AR
4,California,Calif.,CA,US,US-CA


In [6]:
# US overall interest by region
pytrends.build_payload(kw_list=['coronavirus','mask','sanitizer','toilet paper'], geo='US')
region_interest = pytrends.interest_by_region()
region_interest.head()  #Use for standardize interest data across states

,coronavirus,mask,sanitizer,toilet paper
geoName,,,,
Alabama,68,27,2,3
Alaska,64,31,2,3
Arizona,66,27,2,5
Arkansas,67,28,2,3
California,62,32,3,3


In [7]:
# Get historical hourly interest for "coronavirus" of CA from Jan to today
pytrends.get_historical_interest(['coronavirus'], year_start=2020, month_start=1, month_end=5, year_end=2020, geo='US-CA', sleep=0)

HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
The request failed: Google returned a response with code 400.
The request failed: Google returned a response with code 400.


,coronavirus,isPartial
date,,
2020-01-01 00:00:00,6,False
2020-01-01 01:00:00,12,False
2020-01-01 02:00:00,0,False
2020-01-01 03:00:00,0,False
2020-01-01 04:00:00,0,False
...,...,...
2020-04-15 08:00:00,89,False
2020-04-15 09:00:00,85,False
2020-04-15 10:00:00,88,False


In [ ]:
# Get hourly interest for target search terms of US overall and all the states
trend = pytrends.get_historical_interest(['coronavirus','mask','sanitizer','toilet paper'], year_start=2020, month_start=1, month_end=5, year_end=2020, geo='US', sleep=0)
trend['state'] = 'US'
for state in state['geo']:
  interest = pytrends.get_historical_interest(['coronavirus','mask','sanitizer','toilet paper'], year_start=2020, month_start=1, month_end=5, year_end=2020, geo=state, sleep=0)
  interest['state'] = state
  trend = trend.append(interest)

## Data Integration

In [6]:
# Remove the day when the data was collected(partial data) and US overall data
trend = pd.read_csv("trend.csv")
trend = trend[(trend['Date']!='4/13/20') & (trend['state']!='US')]   
trend.head()

,Date,coronavirus,mask,sanitizer,toilet paper,state
85,1/1/20,0,167,1,10,AK
86,1/2/20,0,258,9,24,AK
87,1/3/20,1,227,10,11,AK
88,1/4/20,0,270,8,15,AK
89,1/5/20,0,257,11,146,AK


In [7]:
trend.shape

(3331, 6)

In [8]:
# Data from hospital for each state
hospitalized = pd.read_csv("COVID-19 Hospitalized.csv")
hospitalized.head()

,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,...,hospitalized,total,totalTestResults,posNeg,fips,deathIncrease,hospitalizedIncrease,negativeIncrease,positiveIncrease,totalTestResultsIncrease
0,AK,272.0,7766.0,NaN,NaN,31.0,NaN,NaN,NaN,NaN,...,31.0,8038.0,8038.0,8038.0,2,0.0,0.0,291.0,15.0,306.0
1,AL,3525.0,18058.0,NaN,NaN,437.0,NaN,189.0,NaN,119.0,...,437.0,21583.0,21583.0,21583.0,1,2.0,35.0,0.0,334.0,334.0
2,AR,1280.0,18442.0,NaN,74.0,130.0,NaN,43.0,30.0,39.0,...,130.0,19722.0,19722.0,19722.0,5,3.0,0.0,1090.0,54.0,1144.0
3,AS,0.0,3.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,5.0,3.0,3.0,60,0.0,0.0,0.0,0.0,0.0
4,AZ,3539.0,38570.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,42109.0,42109.0,42109.0,4,7.0,0.0,1433.0,146.0,1579.0


In [9]:
hospitalized.shape

(2112, 25)

In [10]:
# Data about medical resource for each state
medical_resource = pd.read_csv("COVID-19 Medical Resources Population.csv")
medical_resource.head()

,state,total_hospital_beds,total_icu_beds,hospital_bed_occupancy_rate,icu_bed_occupancy_rate,available_hospital_beds,potentially_available_hospital_beds,available_icu_beds,potentially_available_icu_beds,adult_population,...,percentage_of_potentially_available_icu_beds_needed_six_months,percentage_of_total_icu_beds_needed_six_months,icu_beds_needed_twelve_months,percentage_of_available_icu_beds_needed_twelve_months,percentage_of_potentially_available_icu_beds_needed_twelve_months,percentage_of_total_icu_beds_needed_twelve_months,icu_beds_needed_eighteen_months,percentage_of_available_icu_beds_needed_eighteen_months,percentage_of_potentially_available_icu_beds_needed_eighteen_months,percentage_of_total_icu_beds_needed_eighteen_months
0,AK,1583.0,130.0,0.66,0.58,533.0,1058.0,55.0,93.0,552319.0,...,3.34,2.39,155.0,2.82,1.67,1.19,101.0,1.84,1.09,0.78
1,AL,13959.0,1870.0,0.64,0.68,4994.0,9476.0,606.0,1238.0,3748089.0,...,1.83,1.21,1131.0,1.87,0.91,0.60,738.0,1.22,0.60,0.39
2,AR,8428.0,856.0,0.52,0.58,4069.0,6248.0,362.0,609.0,2272226.0,...,2.27,1.61,690.0,1.91,1.13,0.81,450.0,1.24,0.74,0.53
3,AZ,12868.0,1742.0,0.62,0.53,4938.0,8903.0,814.0,1278.0,5187520.0,...,2.47,1.82,1581.0,1.94,1.24,0.91,1031.0,1.27,0.81,0.59
4,CA,68554.0,8131.0,0.67,0.58,22831.0,45692.0,3381.0,5756.0,29868127.0,...,3.04,2.15,8737.0,2.58,1.52,1.07,5698.0,1.69,0.99,0.70


In [11]:
medical_resource.shape

(53, 38)

In [ ]:
# "Stay-home" orders date for each state
intervention = pd.read_csv("intervention.csv")
intervention.head()

In [13]:
# Combine datasets
dataset = trend.merge(hospitalized, 'left', left_on=['Date','state'], right_on=['date','state']) \
            .merge(medical_resource, 'left', left_on='state', right_on='state') \
            .merge(intervention, 'left', left_on='state', right_on='state_name')
dataset.head()

,Date,coronavirus,mask,sanitizer,toilet paper,state,positive,negative,pending,hospitalizedCurrently,...,icu_beds_needed_twelve_months,percentage_of_available_icu_beds_needed_twelve_months,percentage_of_potentially_available_icu_beds_needed_twelve_months,percentage_of_total_icu_beds_needed_twelve_months,icu_beds_needed_eighteen_months,percentage_of_available_icu_beds_needed_eighteen_months,percentage_of_potentially_available_icu_beds_needed_eighteen_months,percentage_of_total_icu_beds_needed_eighteen_months,state_name,order_date
0,1/1/20,0,167,1,10,AK,NaN,NaN,NaN,NaN,...,155.0,2.82,1.67,1.19,101.0,1.84,1.09,0.78,AK,3/28/20
1,1/2/20,0,258,9,24,AK,NaN,NaN,NaN,NaN,...,155.0,2.82,1.67,1.19,101.0,1.84,1.09,0.78,AK,3/28/20
2,1/3/20,1,227,10,11,AK,NaN,NaN,NaN,NaN,...,155.0,2.82,1.67,1.19,101.0,1.84,1.09,0.78,AK,3/28/20
3,1/4/20,0,270,8,15,AK,NaN,NaN,NaN,NaN,...,155.0,2.82,1.67,1.19,101.0,1.84,1.09,0.78,AK,3/28/20
4,1/5/20,0,257,11,146,AK,NaN,NaN,NaN,NaN,...,155.0,2.82,1.67,1.19,101.0,1.84,1.09,0.78,AK,3/28/20


In [14]:
# Fill positive/negative cases with 0 before coronavirus spreaded in the US
dataset['positive'].fillna(0,inplace=True)
dataset['negative'].fillna(0,inplace=True)

In [ ]:
# Create the indicator variable of whether the state begins "stay-home" order
dataset['home'] = (dataset['Date']>=dataset['order_date']).astype('int')

In [16]:
dataset.columns

Index(['Date', 'coronavirus', 'mask', 'sanitizer', 'toilet paper', 'state',
       'positive', 'negative', 'pending', 'hospitalizedCurrently',
       'hospitalizedCumulative', 'inIcuCurrently', 'inIcuCumulative',
       'onVentilatorCurrently', 'onVentilatorCumulative', 'recovered', 'hash',
       'date', 'dateChecked', 'death', 'hospitalized', 'total',
       'totalTestResults', 'posNeg', 'fips', 'deathIncrease',
       'hospitalizedIncrease', 'negativeIncrease', 'positiveIncrease',
       'totalTestResultsIncrease', 'total_hospital_beds', 'total_icu_beds',
       'hospital_bed_occupancy_rate', 'icu_bed_occupancy_rate',
       'available_hospital_beds', 'potentially_available_hospital_beds',
       'available_icu_beds', 'potentially_available_icu_beds',
       'adult_population', 'population_65', 'projected_infected_individuals',
       'proejcted_hospitalized_individuals',
       'projected_individuals_needing_icu_care',
       'hospital_beds_needed_six_months',
       'percentage_of

In [17]:
# Remove the unuseful columns
dataset.drop(columns=['order_date','hash','dateChecked','state_name','date'],inplace=True)
dataset.head()

,Date,coronavirus,mask,sanitizer,toilet paper,state,positive,negative,pending,hospitalizedCurrently,...,percentage_of_total_icu_beds_needed_six_months,icu_beds_needed_twelve_months,percentage_of_available_icu_beds_needed_twelve_months,percentage_of_potentially_available_icu_beds_needed_twelve_months,percentage_of_total_icu_beds_needed_twelve_months,icu_beds_needed_eighteen_months,percentage_of_available_icu_beds_needed_eighteen_months,percentage_of_potentially_available_icu_beds_needed_eighteen_months,percentage_of_total_icu_beds_needed_eighteen_months,home
0,1/1/20,0,167,1,10,AK,0.0,0.0,NaN,NaN,...,2.39,155.0,2.82,1.67,1.19,101.0,1.84,1.09,0.78,0
1,1/2/20,0,258,9,24,AK,0.0,0.0,NaN,NaN,...,2.39,155.0,2.82,1.67,1.19,101.0,1.84,1.09,0.78,0
2,1/3/20,1,227,10,11,AK,0.0,0.0,NaN,NaN,...,2.39,155.0,2.82,1.67,1.19,101.0,1.84,1.09,0.78,0
3,1/4/20,0,270,8,15,AK,0.0,0.0,NaN,NaN,...,2.39,155.0,2.82,1.67,1.19,101.0,1.84,1.09,0.78,0
4,1/5/20,0,257,11,146,AK,0.0,0.0,NaN,NaN,...,2.39,155.0,2.82,1.67,1.19,101.0,1.84,1.09,0.78,0


In [18]:
dataset.shape

(3331, 65)

In [19]:
# Save the final dataset to csv file
dataset.to_csv("final_dataset.csv",index=False)

## Data Cleaning

In [20]:
covid = pd.read_csv("final_dataset.csv")
covid.head()

,date,state,coronavirus,mask,sanitizer,toilet paper,positive,positiveIncrease,negative,negativeIncrease,...,hospitalizedIncrease,total_hospital_beds,available_hospital_beds,hospital_bed_occupancy_rate,total_icu_beds,available_icu_beds,icu_bed_occupancy_rate,adult_population,population_65,home
0,2020-01-01,AK,0,167,1,10,0,0.0,0,0.0,...,0.0,1583,533,0.66,130,55,0.58,552319,74340,0
1,2020-01-02,AK,0,258,9,24,0,0.0,0,0.0,...,0.0,1583,533,0.66,130,55,0.58,552319,74340,0
2,2020-01-03,AK,1,227,10,11,0,0.0,0,0.0,...,0.0,1583,533,0.66,130,55,0.58,552319,74340,0
3,2020-01-04,AK,0,270,8,15,0,0.0,0,0.0,...,0.0,1583,533,0.66,130,55,0.58,552319,74340,0
4,2020-01-05,AK,0,257,11,146,0,0.0,0,0.0,...,0.0,1583,533,0.66,130,55,0.58,552319,74340,0


In [21]:
# Fill the abnormal data in interest as Nan, then fill na by linear method
idx = covid[((covid['date']>='2020-02-25')&(covid['date']<='2020-03-16'))|(covid['date']=='2020-04-01')].index
covid.loc[idx,['mask','sanitizer','toilet paper','coronavirus']] = np.nan
covid['mask'].interpolate(inplace=True)
covid['sanitizer'].interpolate(inplace=True)
covid['toilet paper'].interpolate(inplace=True)
covid['coronavirus'].interpolate(inplace=True)

In [22]:
region = pd.read_csv("interest_region.csv")
region.head()

,geoName,coronavirus,mask,sanitizer,toilet paper,state
0,Alabama,68,27,2,3,AL
1,Alaska,64,31,2,3,AK
2,Arizona,67,27,2,4,AZ
3,Arkansas,67,28,2,3,AR
4,California,62,32,3,3,CA


In [23]:
covid = covid.merge(region, "left", left_on="state", right_on='state')

In [24]:
covid.columns

Index(['date', 'state', 'coronavirus_x', 'mask_x', 'sanitizer_x',
       'toilet paper_x', 'positive', 'positiveIncrease', 'negative',
       'negativeIncrease', 'pending', 'total', 'totalTestResultsIncrease',
       'recovered', 'death', 'deathIncrease', 'hospitalizedCumulative',
       'hospitalizedIncrease', 'total_hospital_beds',
       'available_hospital_beds', 'hospital_bed_occupancy_rate',
       'total_icu_beds', 'available_icu_beds', 'icu_bed_occupancy_rate',
       'adult_population', 'population_65', 'home', 'geoName', 'coronavirus_y',
       'mask_y', 'sanitizer_y', 'toilet paper_y'],
      dtype='object')

In [25]:
# Standarize the interest data across states
covid['coronavirus'] = covid['coronavirus_x']/covid['coronavirus_y']
covid['mask'] = covid['mask_x']/covid['mask_y']
covid['sanitizer'] = covid['sanitizer_x']/covid['sanitizer_y']
covid['toilet paper'] = covid['toilet paper_x']/covid['toilet paper_y']
covid.drop(columns=['coronavirus_x','coronavirus_y','mask_x','mask_y','sanitizer_x','sanitizer_y','toilet paper_x','toilet paper_y'],inplace=True)
covid.head()

,date,state,positive,positiveIncrease,negative,negativeIncrease,pending,total,totalTestResultsIncrease,recovered,...,available_icu_beds,icu_bed_occupancy_rate,adult_population,population_65,home,geoName,coronavirus,mask,sanitizer,toilet paper
0,2020-01-01,AK,0,0.0,0,0.0,0.0,0.0,0.0,0.0,...,55,0.58,552319,74340,0,Alaska,0.000000,5.387097,0.5,3.333333
1,2020-01-02,AK,0,0.0,0,0.0,0.0,0.0,0.0,0.0,...,55,0.58,552319,74340,0,Alaska,0.000000,8.322581,4.5,8.000000
2,2020-01-03,AK,0,0.0,0,0.0,0.0,0.0,0.0,0.0,...,55,0.58,552319,74340,0,Alaska,0.015625,7.322581,5.0,3.666667
3,2020-01-04,AK,0,0.0,0,0.0,0.0,0.0,0.0,0.0,...,55,0.58,552319,74340,0,Alaska,0.000000,8.709677,4.0,5.000000
4,2020-01-05,AK,0,0.0,0,0.0,0.0,0.0,0.0,0.0,...,55,0.58,552319,74340,0,Alaska,0.000000,8.290323,5.5,48.666667


## Panel Model Building

In [30]:
from linearmodels.panel import PanelOLS
import statsmodels.api as sm
from sklearn import linear_model
import statsmodels.api as sm

- Fixed entity effect: catch unobserved variables that change with entity but remain the same all the time (state specific politics, demographics, economics, etc) <br>
- Fixed time effect: catch unobserved varaibales that change with time but remain the same across states (global pandemic trend)

In [31]:
df = covid.copy()
df['date'] = df['date'].astype("datetime64[ns]")
df.head()

,date,state,positive,positiveIncrease,negative,negativeIncrease,pending,total,totalTestResultsIncrease,recovered,...,population_65,home,geoName,coronavirus,mask,sanitizer,toilet paper,fips,cases,deaths
0,2020-01-01,AK,0,0.0,0,0.0,0.0,0.0,0.0,0.0,...,74340,0,Alaska,0.000000,5.387097,0.5,3.333333,0.0,0.0,0.0
1,2020-01-02,AK,0,0.0,0,0.0,0.0,0.0,0.0,0.0,...,74340,0,Alaska,0.000000,8.322581,4.5,8.000000,0.0,0.0,0.0
2,2020-01-03,AK,0,0.0,0,0.0,0.0,0.0,0.0,0.0,...,74340,0,Alaska,0.015625,7.322581,5.0,3.666667,0.0,0.0,0.0
3,2020-01-04,AK,0,0.0,0,0.0,0.0,0.0,0.0,0.0,...,74340,0,Alaska,0.000000,8.709677,4.0,5.000000,0.0,0.0,0.0
4,2020-01-05,AK,0,0.0,0,0.0,0.0,0.0,0.0,0.0,...,74340,0,Alaska,0.000000,8.290323,5.5,48.666667,0.0,0.0,0.0


In [32]:
# Create lags for order_date(home) and search interest 
df1=df.copy()
df1['home']=df1.sort_values(['date']).groupby(['state'])['home'].shift(15)
df1['mask']=df1.sort_values(['date']).groupby(['state'])['mask'].shift(15)
df1['sanitizer']=df1.sort_values(['date']).groupby(['state'])['sanitizer'].shift(15)
df1['toilet paper']=df1.sort_values(['date']).groupby(['state'])['toilet paper'].shift(15)

In [33]:
# Only consider the data after 2020-03-01
df1 = df1[df1['date']>"2020-03-01"]

In [34]:
df1 = df1.set_index(["state","date"])
df1.head()

positive  positiveIncrease  negative  negativeIncrease  \
state date                                                                 
AK    2020-03-11         0               0.0        46              23.0   
      2020-03-12         0               0.0        46               0.0   
      2020-03-13         1               1.0        59              13.0   
      2020-03-14         1               0.0       143              84.0   
      2020-03-15         1               0.0       143               0.0   

                  pending  total  totalTestResultsIncrease  recovered  death  \
state date                                                                     
AK    2020-03-11     14.0   60.0                      23.0        0.0    0.0   
      2020-03-12     14.0   60.0                       0.0        0.0    0.0   
      2020-03-13      0.0   60.0                      14.0        0.0    0.0   
      2020-03-14      0.0  144.0                      84.0        0.0    0.0   
      2020-03-15      0.0  144.0                       0.0        0.0    0.0   

                  deathIncrease  ...  population_65  home  geoName  \
state date                       ...                                 
AK    2020-03-11            0.0  ...          74340   0.0   Alaska   
      2020-03-12            0.0  ...          74340   0.0   Alaska   
      2020-03-13            0.0  ...          74340   0.0   Alaska   
      2020-03-14            0.0  ...          74340   0.0   Alaska   
      2020-03-15            0.0  ...          74340   0.0   Alaska   

                  coronavirus      mask  sanitizer  toilet paper  fips  cases  \
state date                                                                      
AK    2020-03-11    10.536458  5.483871        6.0      2.333333   0.0    0.0   
      2020-03-12    10.413194  5.516129        3.0      3.666667   2.0    1.0   
      2020-03-13    10.289931  5.806452        2.0      3.666667   2.0    1.0   
      2020-03-14    10.166667  5.193548        4.0      4.000000   2.0    1.0   
      2020-03-15    10.043403  5.709677        3.0     44.000000   2.0    1.0   

                  deaths  
state date                
AK    2020-03-11     0.0  
      2020-03-12     0.0  
      2020-03-13     0.0  
      2020-03-14     0.0  
      2020-03-15     0.0  

[5 rows x 29 columns]

In [35]:
df1.shape

(1064, 29)

In [36]:
df1.columns

Index(['positive', 'positiveIncrease', 'negative', 'negativeIncrease',
       'pending', 'total', 'totalTestResultsIncrease', 'recovered', 'death',
       'deathIncrease', 'hospitalizedCumulative', 'hospitalizedIncrease',
       'total_hospital_beds', 'available_hospital_beds',
       'hospital_bed_occupancy_rate', 'total_icu_beds', 'available_icu_beds',
       'icu_bed_occupancy_rate', 'adult_population', 'population_65', 'home',
       'geoName', 'coronavirus', 'mask', 'sanitizer', 'toilet paper', 'fips',
       'cases', 'deaths'],
      dtype='object')

In [37]:
# Standardize variables
df1['hospitalizedIncrease_per'] = df1['hospitalizedIncrease']/df1['adult_population']
df1['available_hospital_beds_per'] = df1['available_hospital_beds']/df1['adult_population']
df1['available_icu_beds_per'] = df1['available_icu_beds']/df1['adult_population']
df1['coronavirus_per'] = df1['coronavirus']/df1['adult_population']
df1['mask_per'] = df1['mask']/df1['adult_population']
df1['sanitizer_per'] = df1['sanitizer']/df1['adult_population']
df1['toilet paper_per'] = df1['toilet paper']/df1['adult_population']
df1['deathIncrease_per'] = df1['deathIncrease']/df1['adult_population']
df1['positiveIncrease_per'] = df1['positiveIncrease']/df1['adult_population']

In [38]:
df1.head()

positive  positiveIncrease  negative  negativeIncrease  \
state date                                                                 
AK    2020-03-11         0               0.0        46              23.0   
      2020-03-12         0               0.0        46               0.0   
      2020-03-13         1               1.0        59              13.0   
      2020-03-14         1               0.0       143              84.0   
      2020-03-15         1               0.0       143               0.0   

                  pending  total  totalTestResultsIncrease  recovered  death  \
state date                                                                     
AK    2020-03-11     14.0   60.0                      23.0        0.0    0.0   
      2020-03-12     14.0   60.0                       0.0        0.0    0.0   
      2020-03-13      0.0   60.0                      14.0        0.0    0.0   
      2020-03-14      0.0  144.0                      84.0        0.0    0.0   
      2020-03-15      0.0  144.0                       0.0        0.0    0.0   

                  deathIncrease  ...  deaths  hospitalizedIncrease_per  \
state date                       ...                                     
AK    2020-03-11            0.0  ...     0.0                       0.0   
      2020-03-12            0.0  ...     0.0                       0.0   
      2020-03-13            0.0  ...     0.0                       0.0   
      2020-03-14            0.0  ...     0.0                       0.0   
      2020-03-15            0.0  ...     0.0                       0.0   

                  available_hospital_beds_per  available_icu_beds_per  \
state date                                                              
AK    2020-03-11                     0.000965                  0.0001   
      2020-03-12                     0.000965                  0.0001   
      2020-03-13                     0.000965                  0.0001   
      2020-03-14                     0.000965                  0.0001   
      2020-03-15                     0.000965                  0.0001   

                  coronavirus_per  mask_per  sanitizer_per  toilet paper_per  \
state date                                                                     
AK    2020-03-11         0.000019  0.000010       0.000011          0.000004   
      2020-03-12         0.000019  0.000010       0.000005          0.000007   
      2020-03-13         0.000019  0.000011       0.000004          0.000007   
      2020-03-14         0.000018  0.000009       0.000007          0.000007   
      2020-03-15         0.000018  0.000010       0.000005          0.000080   

                  deathIncrease_per  positiveIncrease_per  
state date                                                 
AK    2020-03-11                0.0              0.000000  
      2020-03-12                0.0              0.000000  
      2020-03-13                0.0              0.000002  
      2020-03-14                0.0              0.000000  
      2020-03-15                0.0              0.000000  

[5 rows x 38 columns]

## Model Comparison

### FE Model 1: entity_effects=True, time_effects=True

In [40]:
# predict positiveIncrease
exog = df1[["hospitalizedIncrease_per", "available_hospital_beds_per", 
            "available_icu_beds_per", "icu_bed_occupancy_rate", "home", 
            "coronavirus_per", "mask_per", "sanitizer_per", "toilet paper_per"]]
fe = PanelOLS(df1['positiveIncrease_per'], exog, entity_effects=True, time_effects=True, drop_absorbed=True)
fe = fe.fit()
print(fe)

                           PanelOLS Estimation Summary                            
Dep. Variable:     positiveIncrease_per   R-squared:                        0.1368
Estimator:                     PanelOLS   R-squared (Between):              0.3642
No. Observations:                  1064   R-squared (Within):               0.1202
Date:                  Wed, Apr 15 2020   R-squared (Overall):              0.2308
Time:                          05:36:01   Log-likelihood                    8757.3
Cov. Estimator:              Unadjusted                                           
                                          F-statistic:                      26.195
Entities:                            40   P-value                           0.0000
Avg Obs:                         26.600   Distribution:                   F(6,992)
Min Obs:                         15.000                                           
Max Obs:                         27.000   F-statistic (robust):             26.195
    

### FE Model 2: entity_effects=False, time_effects=True

In [41]:
# predict positiveIncrease
exog = df1[["hospitalizedIncrease_per", "available_hospital_beds_per", 
            "available_icu_beds_per", "icu_bed_occupancy_rate", "home", 
            "coronavirus_per", "mask_per", "sanitizer_per", "toilet paper_per"]]
fe = PanelOLS(df1['positiveIncrease_per'], exog, entity_effects=False, time_effects=True, drop_absorbed=True)
fe = fe.fit()
print(fe)

                           PanelOLS Estimation Summary                            
Dep. Variable:     positiveIncrease_per   R-squared:                        0.2439
Estimator:                     PanelOLS   R-squared (Between):             -1.6713
No. Observations:                  1064   R-squared (Within):               0.1588
Date:                  Wed, Apr 15 2020   R-squared (Overall):             -0.7720
Time:                          05:36:03   Log-likelihood                    8513.0
Cov. Estimator:              Unadjusted                                           
                                          F-statistic:                      36.835
Entities:                            40   P-value                           0.0000
Avg Obs:                         26.600   Distribution:                  F(9,1028)
Min Obs:                         15.000                                           
Max Obs:                         27.000   F-statistic (robust):             36.835
    

### FE Model 3: entity_effects=True, time_effects=False (Our Final Model)

In [42]:
# predict positiveIncrease
exog = df1[["hospitalizedIncrease_per", "available_hospital_beds_per", 
            "available_icu_beds_per", "icu_bed_occupancy_rate", "home", 
            "coronavirus_per", "mask_per", "sanitizer_per", "toilet paper_per"]]
fe = PanelOLS(df1['positiveIncrease_per'], exog, entity_effects=True, time_effects=False, drop_absorbed=True)
fe = fe.fit(cov_type='clustered', cluster_entity=True)
print(fe)

                           PanelOLS Estimation Summary                            
Dep. Variable:     positiveIncrease_per   R-squared:                        0.2984
Estimator:                     PanelOLS   R-squared (Between):             -0.0424
No. Observations:                  1064   R-squared (Within):               0.2984
Date:                  Wed, Apr 15 2020   R-squared (Overall):              0.1133
Time:                          05:36:05   Log-likelihood                    8668.2
Cov. Estimator:               Clustered                                           
                                          F-statistic:                      72.178
Entities:                            40   P-value                           0.0000
Avg Obs:                         26.600   Distribution:                  F(6,1018)
Min Obs:                         15.000                                           
Max Obs:                         27.000   F-statistic (robust):             10.425
    

### RE Model 

In [43]:
# predict positiveIncrease
exog = df1[["hospitalizedIncrease_per", "available_hospital_beds_per", 
            "available_icu_beds_per", "icu_bed_occupancy_rate", "home", 
            "coronavirus_per", "mask_per", "sanitizer_per", "toilet paper_per"]]
fe = PanelOLS(df1['positiveIncrease_per'], exog, entity_effects=False, time_effects=False, drop_absorbed=True)
fe = fe.fit()
print(fe)

                           PanelOLS Estimation Summary                            
Dep. Variable:     positiveIncrease_per   R-squared:                        0.4490
Estimator:                     PanelOLS   R-squared (Between):              0.6332
No. Observations:                  1064   R-squared (Within):               0.2882
Date:                  Wed, Apr 15 2020   R-squared (Overall):              0.4490
Time:                          05:36:14   Log-likelihood                    8444.9
Cov. Estimator:              Unadjusted                                           
                                          F-statistic:                      95.518
Entities:                            40   P-value                           0.0000
Avg Obs:                         26.600   Distribution:                  F(9,1055)
Min Obs:                         15.000                                           
Max Obs:                         27.000   F-statistic (robust):             95.518
    

### Pooled Model (as a benchmark)

In [44]:
# predict positiveIncrease
X = df1[["hospitalizedIncrease_per", "available_hospital_beds_per", 
            "available_icu_beds_per", "icu_bed_occupancy_rate", "home", 
            "coronavirus_per", "mask_per", "sanitizer_per", "toilet paper_per"]]
y = df1['positiveIncrease_per']
pool = sm.OLS(y, X)
result = pool.fit()
print(result.summary())

                                  OLS Regression Results                                 
Dep. Variable:     positiveIncrease_per   R-squared (uncentered):                   0.449
Model:                              OLS   Adj. R-squared (uncentered):              0.444
Method:                   Least Squares   F-statistic:                              95.52
Date:                  Wed, 15 Apr 2020   Prob (F-statistic):                   5.22e-130
Time:                          05:36:58   Log-Likelihood:                          8444.9
No. Observations:                  1064   AIC:                                 -1.687e+04
Df Residuals:                      1055   BIC:                                 -1.683e+04
Df Model:                             9                                                  
Covariance Type:              nonrobust                                                  
                                  coef    std err          t      P>|t|      [0.025      0.975]
----

## Thanks for reading!
Team Disinfectant Wipes

2020-04-15